In [1]:
%matplotlib inline

In [2]:
# Import Dependencies
import json
import requests
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import scipy.stats as st


In [3]:
# Import csv file and make DataFrame
uber_data = Path("../Resources/Uber_Project_CSV.csv")
uber_df = pd.read_csv(uber_data)
uber_df.head()

,pickup_dt,borough,pickups,spd,vsb,temp,dewp,slp,pcp01,pcp06,pcp24,sd,hday
0,1/1/2015 1:00,Bronx,152,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
1,1/1/2015 1:00,Brooklyn,1519,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
2,1/1/2015 1:00,EWR,0,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
3,1/1/2015 1:00,Manhattan,5258,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y
4,1/1/2015 1:00,Queens,405,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0,Y


In [4]:
#Import URL and API CSV file
url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/new%2520york%2520city/2015-01-01/2015-06-30?elements=datetime%2CdatetimeEpoch%2Ctemp%2Ctempmax%2Ctempmin%2Cprecip%2Cwindspeed%2Cwindgust%2Cfeelslike%2Cfeelslikemax%2Cfeelslikemin%2Cpressure%2Cstations%2Cdegreedays%2Caccdegreedays&include=fcst%2Cobs%2Chistfcst%2Cstats%2Chours&key=9R8U63UTPUJVYD92WRCVMZZXB&options=preview&contentType=json"
weather_api = Path("../Resources/NYC_Weather_API_data.csv")
weather_df= pd.read_csv(weather_api)
weather_df.head()


,name,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,solarenergy,uvindex,severerisk,sunrise,sunset,moonphase,conditions,description,icon,stations
0,new york city,1/1/2015,38.7,27.3,32.8,35.8,17.1,25.6,9.6,37.7,...,9.9,5,NaN,2015-01-01T07:20:09,2015-01-01T16:39:07,0.38,Clear,Clear conditions throughout the day.,clear-day,"72505394728,KLGA,KNYC,72503014732"
1,new york city,1/2/2015,42.2,36.0,38.6,36.5,28.1,32.2,16.8,41.0,...,9.5,5,NaN,2015-01-02T07:20:15,2015-01-02T16:39:58,0.41,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"72505394728,KLGA,KNYC,72503014732"
2,new york city,1/3/2015,40.7,33.5,35.8,35.0,25.9,30.2,26.1,69.4,...,5.2,3,NaN,2015-01-03T07:20:19,2015-01-03T16:40:50,0.45,"Snow, Rain, Partially cloudy",Partly cloudy throughout the day with rain or ...,rain,"72505394728,KLGA,KNYC,72503014732"
3,new york city,1/4/2015,55.4,41.0,48.2,55.4,36.6,46.6,44.6,87.4,...,3.8,3,NaN,2015-01-04T07:20:20,2015-01-04T16:41:44,0.50,"Rain, Overcast",Cloudy skies throughout the day with rain.,rain,"72505394728,KLGA,KNYC,72503014732"
4,new york city,1/5/2015,50.3,21.8,36.3,50.3,11.4,27.8,12.2,37.1,...,9.9,5,NaN,2015-01-05T07:20:19,2015-01-05T16:42:40,0.52,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"72505394728,KLGA,KNYC,72503014732"


In [37]:
# 1 hour, 6 hour, 24 hour - is there a difference
# Difference between rainy  and snowy day pickups? - Caleb
rain_data_df = uber_df[["pickup_dt","borough", "pickups", "pcp01", "pcp06", "pcp24"]]
short_rain_df = rain_data_df[(rain_data_df["pcp01"] > 0) & (rain_data_df["pcp06"] == 0) & (rain_data_df["pcp24"] == 0)]
short_rain_df.groupby("borough").sum("pickups")

,pickups,pcp01,pcp06,pcp24
borough,,,,
Bronx,8584,8.6855,0.0,0.0
Brooklyn,94105,8.6855,0.0,0.0
EWR,1,8.6855,0.0,0.0
Manhattan,447622,8.6855,0.0,0.0
Queens,53488,8.6855,0.0,0.0
Staten Island,275,8.6855,0.0,0.0


In [36]:
medium_rain_df = rain_data_df[(rain_data_df["pcp06"] > 0) & (rain_data_df["pcp01"] == 0) & (rain_data_df["pcp24"] == 0)]
medium_rain_df.groupby("borough").sum("pickups")

,pickups,pcp01,pcp06,pcp24
borough,,,,
Bronx,21022,0.0,38.691,0.0
Brooklyn,234293,0.0,38.691,0.0
EWR,5,0.0,38.691,0.0
Manhattan,917997,0.0,38.691,0.0
Queens,130854,0.0,38.691,0.0
Staten Island,681,0.0,38.691,0.0


In [35]:
heavy_rain_df = rain_data_df[(rain_data_df["pcp24"] > 0) & (rain_data_df["pcp01"] == 0) & (rain_data_df["pcp06"] == 0)]
heavy_rain_df.groupby(["borough"]).sum("pickups")

,pickups,pcp01,pcp06,pcp24
borough,,,,
Bronx,50038,0.0,0.0,239.414167
Brooklyn,514271,0.0,0.0,239.414167
EWR,31,0.0,0.0,239.414167
Manhattan,2433263,0.0,0.0,239.414167
Queens,318784,0.0,0.0,239.414167
Staten Island,1513,0.0,0.0,239.414167


In [38]:
snow_data_df = uber_df[["pickup_dt", "borough", "pickups", "sd"]]
snow_df = snow_data_df[(snow_data_df["sd"] > 0)]
snow_df.groupby("borough").sum("pickups")


,pickups,sd
borough,,
Bronx,53225,11016.0
Brooklyn,623687,11016.0
EWR,27,11016.0
Manhattan,3149007,11016.0
Queens,351742,11016.0
Staten Island,1643,11016.0


In [43]:
no_precipitation_df = uber_df[(uber_df["pcp01"] == 0) & (uber_df["pcp06"] == 0) & (uber_df["pcp24"] == 0) & (uber_df["sd"] == 0)]
no_precipitation_df.valuecount()

AttributeError: 'DataFrame' object has no attribute 'valuecount'

In [6]:
# How does the relative temperature and humidity affect pickups? - Caleb

In [7]:
# Does Wind Speed affect the number of pickups? How does wind with or without precipitation affect it? - Ryan
wind_df= uber_df[["pickup_dt", "borough", "pickups", "spd"]]

wind_df.head()


,pickup_dt,borough,pickups,spd
0,1/1/2015 1:00,Bronx,152,5.0
1,1/1/2015 1:00,Brooklyn,1519,5.0
2,1/1/2015 1:00,EWR,0,5.0
3,1/1/2015 1:00,Manhattan,5258,5.0
4,1/1/2015 1:00,Queens,405,5.0


In [8]:
#Does different boroughs? Ryan

In [9]:
# Time of day effect pickups? - Fidel

In [10]:
#Difference in Weekdays vs Weekends? - James

In [11]:
#How do Holidays affect pickups? - Fidel Carillo Average holiday pickups vs non holiday with rain and vs non holiday without Difference in Holiday pickups? - Fidel

In [12]:
# Correlation and Regression

## Analysis